* Add `EDS_TOKEN` to the `.env` file

In [ ]:
import { logError } from "./logError.js";
import { reloadEnv } from "./reloadEnv.js"
await reloadEnv();

const token = Deno.env.get("EDS_TOKEN");

const org = 'adobecom';
const repo = 'cc';
const merchCardPath = '/cc-shared/fragments/merch/products/catalog/merch-card';
const format = 'html';  // 'md' or 'html'


* Test if the token is working

In [ ]:
const response = await fetch(`https://admin.hlx.page/status/${org}/${repo}/main/`, {
  headers: {
    'x-auth-token': token
  }
});

if (!response.ok) {
  logError(response);
}

console.log(response.status);

* Query the list of files with Helix API

In [ ]:
import { scheduler } from "node:timers/promises";

const response = await fetch(`https://admin.hlx.page/status/${org}/${repo}/main/*`, {
  method: 'POST',
  headers: {
    'x-auth-token': token,
    'Content-Type': 'application/json'
  },

  body: JSON.stringify({
    "select": [
      'live'
    ],
    "paths": [
      `${merchCardPath}/*`
    ],
    "pathsOnly": true
  })
});

if (!response.ok) {
  logError(response);
}

const data = await response.json();
console.log(`Job link: ${data.links.self}`);

const jobLink = data.links.self;

let jobDetails;

let retry = 10;
while (retry > 0) {
  const responseState = await fetch(jobLink, {
    headers: {
      'x-auth-token': token
    }
  });

  if (!responseState.ok) {
    logError(responseState);
  }

  const dataState = await responseState.json();

  console.log(`Job state: ${dataState.state}`);

  if (dataState.state === 'stopped') {
    jobDetails = dataState.links.details;
    break;
  }

  await scheduler.wait(5000)
  retry -= 1;
}

const responseDetail = await fetch(jobDetails, {
  headers: {
    'x-auth-token': token
  }
});

if (!responseDetail.ok) {
  logError(responseDetail);
}

const dataDetail = await responseDetail.json();
const fileList = dataDetail.data.resources['live'];

console.log(`page and asset count: ${fileList.length}`);

const fileName = `${repo}_filelist.txt`;
await Deno.writeTextFile(fileName, fileList.join('\n'));
console.log(`\nOutput saved to: ${fileName}`);


In [ ]:
import { ensureDir } from "https://deno.land/std@0.208.0/fs/ensure_dir.ts";
import { dirname } from "https://deno.land/std@0.208.0/path/mod.ts";

const ext = {'md': '.md', 'html': '.plain.html'}[format];

let successCount = 0;
let failureCount = 0;

if (ext) {
  for (let i = 0; i < fileList.length; i++) {
    const filePath = fileList[i];
    const response = await fetch(`https://main--${repo}--${org}.aem.live${filePath}${ext}`);
    if (!response.ok) {
      logError(response);
      failureCount++;
    } else {
      const data = await response.text();
      const localPath = `./${format}${filePath}${ext}`;
      await ensureDir(dirname(localPath));
      await Deno.writeTextFile(localPath, data);
      successCount++;
    }
  }
}

console.log(`\nResults: ${successCount} succeeded, ${failureCount} failed out of ${fileList.length} total`);
